# Required

In [1]:
import os
import pandas as pd
import zipfile
import numpy as np
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split

import torch.optim as optim
from torch.optim import SGD
from tqdm import tqdm
import torchvision.transforms as T

import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.ops import box_iou

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_dir = "/content/gdrive/MyDrive/ColabNotebooks/TrainIJCNN2013.zip"
test_dir = "/content/gdrive/MyDrive/ColabNotebooks/TestIJCNN2013.zip"
annotations_file = "/content/gdrive/MyDrive/ColabNotebooks/gt.txt"

In [ ]:
local_zip = 'TrainIJCNN2013.zip'
zip_ref = zipfile.ZipFile(train_dir, 'r')
zip_ref.extractall('/content')
zip_ref.close()

local_zip = 'TestIJCNN2013.zip'
zip_ref = zipfile.ZipFile(test_dir, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
train_dir = 'TrainIJCNN2013'
test_dir = 'TestIJCNN2013Download'
shutil.copy(annotations_file, train_dir)
annotations_file = "/content/TrainIJCNN2013/gt.txt"

In [ ]:
# Έλεγχος περιεχομένων του αρχείου
with open(annotations_file, 'r') as file:
    for i in range(5):
        print(file.readline())

# Φόρτωση του αρχείου με το σωστό διαχωριστικό
annotations = pd.read_csv(annotations_file, delimiter=',', header=0)

# Εμφάνιση των πρώτων γραμμών και της δομής του DataFrame
print(annotations.head())
print(annotations.columns)
print(annotations.shape)

filename,x1,y1,x2,y2,class

/content/TrainIJCNN2013/00000.ppm,774,411,815,446,11

/content/TrainIJCNN2013/00001.ppm,983,388,1024,432,40

/content/TrainIJCNN2013/00001.ppm,386,494,442,552,38

/content/TrainIJCNN2013/00001.ppm,973,335,1031,390,13

                            filename   x1   y1    x2   y2  class
0  /content/TrainIJCNN2013/00000.ppm  774  411   815  446     11
1  /content/TrainIJCNN2013/00001.ppm  983  388  1024  432     40
2  /content/TrainIJCNN2013/00001.ppm  386  494   442  552     38
3  /content/TrainIJCNN2013/00001.ppm  973  335  1031  390     13
4  /content/TrainIJCNN2013/00002.ppm  892  476  1006  592     39
Index(['filename', 'x1', 'y1', 'x2', 'y2', 'class'], dtype='object')
(852, 6)


In [ ]:
# Βεβαιώνουμε ότι οι στήλες είναι αριθμητικές
train_annotations['x1'] = pd.to_numeric(train_annotations['x1'], errors='coerce')
train_annotations['y1'] = pd.to_numeric(train_annotations['y1'], errors='coerce')
train_annotations['x2'] = pd.to_numeric(train_annotations['x2'], errors='coerce')
train_annotations['y2'] = pd.to_numeric(train_annotations['y2'], errors='coerce')

# Αφαιρούμε τυχόν γραμμές με μη αριθμητικά δεδομένα ή κενές τιμές
train_annotations = train_annotations.dropna()

# Επαληθεύουμε τη μορφή των δεδομένων
print(train_annotations.dtypes)
print(train_annotations.head())


filename    object
x1           int64
y1           int64
x2           int64
y2           int64
class        int64
dtype: object
                            filename   x1   y1    x2   y2  class
0  /content/TrainIJCNN2013/00000.ppm  774  411   815  446     11
1  /content/TrainIJCNN2013/00001.ppm  983  388  1024  432     40
2  /content/TrainIJCNN2013/00001.ppm  386  494   442  552     38
3  /content/TrainIJCNN2013/00001.ppm  973  335  1031  390     13
4  /content/TrainIJCNN2013/00002.ppm  892  476  1006  592     39


In [ ]:
# Διάσπαση σε train και test με βάση τα υπάρχοντα directories
train_annotations = annotations  # Χρησιμοποιούμε τα δεδομένα εκπαίδευσης ως έχουν
test_annotations_file = os.path.join(test_dir, annotations_file)
test_annotations = pd.read_csv(annotations_file, delimiter=',', header=0)
test_annotations.columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id']

# Προεπισκόπηση δεδομένων
print(f"Train annotations: {len(train_annotations)}")
print(f"Test annotations: {len(test_annotations)}")

Train annotations: 852
Test annotations: 852


In [ ]:
class GTSDBDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.annotations = annotations_file
        self.img_dir = img_dir
        self.transform = transform # Προαιρετικοί μετασχηματισμοί (π.χ. κανονικοποίηση)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Φόρτωση εικόνας
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")

        # Φόρτωση bounding box και ετικέτας
        boxes = self.annotations.iloc[idx, 1:5].values.astype(float)  # Μετατροπή σε float
        boxes = torch.tensor(boxes, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor([self.annotations.iloc[idx, 5]], dtype=torch.int64)

        target = {"boxes": boxes, "labels": label}

        # Εφαρμογή μετασχηματισμών
        if self.transform:
            image = self.transform(image)

        return image, target

In [ ]:
# Φόρτωση GTSDB Dataset

transform = T.Compose([
    T.ToTensor(), # Μετατροπή εικόνας σε Tensor
    T.Resize((300, 300)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Κανονικοποίηση
])

train_dataset = GTSDBDataset(train_annotations, train_dir, transform)
test_dataset = GTSDBDataset(test_annotations, test_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
# Φόρτωση COCO Dataset

coco_train_dataset = CocoDetection(root=train_dir, annFile=train_annotations, transform=transform)

coco_train_loader = DataLoader(coco_train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

print(f"COCO Dataset Loaded: {len(coco_train_dataset)} images")

# Faster-RCNN

In [ ]:
# Φόρτωση του Faster R-CNN με βάση MobileNet V3
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Προσαρμογή του αριθμού κατηγοριών
num_classes = 44  # 43 κατηγορίες + background
in_features = model.roi_heads.box_predictor.cls_score.in_features  # Είσοδος του τελευταίου επιπέδου
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Μεταφορά του μοντέλου στη GPU (αν υπάρχει)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print("Το Faster-RCNN προσαρμόστηκε για 44 κατηγορίες και μεταφέρθηκε στη συσκευή:", device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Το Faster-RCNN προσαρμόστηκε για 44 κατηγορίες και μεταφέρθηκε στη συσκευή: cuda


In [ ]:
# Ορισμός υπερπαραμέτρων
num_epochs = 10
learning_rate = 0.005
momentum = 0.9
weight_decay = 0.0005

# Ορισμός του optimizer
optimizer = SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Training Loop
for epoch in range(num_epochs):
    model.train()  # Το μοντέλο σε training mode
    epoch_loss = 0  # Παρακολούθηση της απώλειας
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")  # Γραμμή προόδου

    for images, targets in pbar:
        # Μεταφορά των δεδομένων στη συσκευή (GPU ή CPU)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Μηδενισμός των gradients
        optimizer.zero_grad()

        # Υπολογισμός της απώλειας (loss)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        # Οπισθοδιάδοση (backpropagation)
        losses.backward()

        # Ενημέρωση των βαρών
        optimizer.step()

        # Ενημέρωση της γραμμής προόδου
        pbar.set_postfix(loss=losses.item())

    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

# Αποθήκευση του εκπαιδευμένου μοντέλου
torch.save(model.state_dict(), "faster_rcnn_gtsdb.pth")
print("Το μοντέλο αποθηκεύτηκε ως faster_rcnn_gtsdb.pth")

Epoch 1/10: 100%|██████████| 213/213 [02:39<00:00,  1.33it/s, loss=nan]


Epoch 1 Loss: nan


Epoch 2/10: 100%|██████████| 213/213 [02:32<00:00,  1.40it/s, loss=nan]


Epoch 2 Loss: nan


Epoch 3/10: 100%|██████████| 213/213 [02:32<00:00,  1.40it/s, loss=nan]


...

Epoch 8 Loss: nan


Epoch 9/10: 100%|██████████| 213/213 [02:31<00:00,  1.40it/s, loss=nan]


Epoch 9 Loss: nan


Epoch 10/10: 100%|██████████| 213/213 [02:32<00:00,  1.40it/s, loss=nan]


Epoch 10 Loss: nan
Το μοντέλο αποθηκεύτηκε ως faster_rcnn_gtsdb.pth


In [ ]:
# Λειτουργία αξιολόγησης (inference mode)
model.eval()

# Αποθήκευση των αποτελεσμάτων
all_preds = []
all_targets = []
print(all_targets[:5])
print(all_preds[:5])

with torch.no_grad():
    for images, targets in tqdm(test_loader, desc="Evaluating"):
        # Μεταφορά των δεδομένων στη συσκευή (GPU/CPU)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Προβλέψεις του μοντέλου
        predictions = model(images)

        # Αποθήκευση targets και predictions για αξιολόγηση
        all_preds.extend(predictions)
        all_targets.extend(targets)

Evaluating: 100%|██████████| 213/213 [01:17<00:00,  2.75it/s]


In [ ]:
# Υπολογισμός mAP
def calculate_map(preds, targets, iou_threshold=0.3):
    """
    Υπολογισμός του Mean Average Precision (mAP) για το test set.
    """
    total_ap = []

    for pred, target in zip(preds, targets):
        if len(pred["boxes"]) == 0 or len(target["boxes"]) == 0:
            continue

        # Υπολογισμός IoU (Intersection over Union)
        ious = box_iou(pred["boxes"], target["boxes"])

        # Καταμέτρηση σωστών ανιχνεύσεων
        matches = (ious > iou_threshold).sum().item()

        # Υπολογισμός precision/recall (απλοποιημένο)
        precision = matches / len(pred["boxes"])
        recall = matches / len(target["boxes"])
        ap = precision * recall
        total_ap.append(ap)

    return np.mean(total_ap)

In [ ]:
# Υπολογισμός mAP για το test set
map_score = calculate_map(all_preds, all_targets)
print(f"Mean Average Precision (mAP): {map_score:.4f}")

Mean Average Precision (mAP): nan


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
